# Forward Error Correction Decoding

In [1]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import xsdfec

ol = Overlay('bitstream/book_design.bit')

In [2]:
%store -r llrs tx_buffer
initial_data = tx_buffer

The SD-FEC decoder accepts data in the form *fix6_2*. This data is symmetrically saturated meaning the range is from -7.75 to 7.75. Any values outside of this range are saturated to these maximum and minimum values. If too much saturation is occurring, the performance of the decoder may be affected. To avoid this, the data can be scaled by a number less than 1.

Additionally, this data must be sign extended to the nearest byte. Taking the limits as an example, the format

| Decimal Value | Binary Value | Unsigned Value |
| -----------   | -----------  | -----------    |
| 7.75          |  00011111    | 31             |
| -7.75         |  11100001    | 225            |

A Python function *format_llr* for formatting the data correctly is used to convert the soft data.

In [3]:
def format_llr(v):
    # Perform symmetric saturation
    if v > 7.75:
        v = 7.75
    elif v < -7.75:
        v = -7.75

    v = int(v * pow(2,2)) # Bit shift to expose 2 fractional

    # Convert to binary (2's complement)
    if v >= 0:
        b = '{0:06b}'.format(v)
    else:
        b = '{0:06b}'.format(pow(2,6)+v)

    # Sign extend to 8-bits
    if b[0] == '0': # Number is positive
        b_ex = '00' + b
    else: # Number is negative
        b_ex = '11' + b

    return int(b_ex,2)

Format all the LLR values for the decoder

In [4]:
llrs_formatted = []
for llr in llrs:
    llrs_formatted.append(format_llr(llr))

Load the LDPC codes into the decoder

In [5]:
fec = ol.ldpc_decoder.sd_fec
ldpc_params = fec.available_ldpc_params()

fec.CORE_ORDER = 0                         # In order outputs
fec.CORE_AXIS_ENABLE = 0                   # Ensure FEC is disabled (000000)

sc_offset = 0
la_offset = 0
qc_offset = 0
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Status','Code ID','SC Offset','LA Offset','QC Offset','Code Name'))
print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('------','------','------','------','------','------'))
for code_id in range(len(ldpc_params)):
    
    code_name = ldpc_params[code_id]
    table_sizes = fec.share_table_size(code_name)

    fec.add_ldpc_params(code_id, sc_offset, la_offset, qc_offset, code_name)
    print("{:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Loaded',code_id, sc_offset, la_offset, qc_offset, code_name))
    
    sc_offset += table_sizes['sc_size']
    la_offset += table_sizes['la_size']
    qc_offset += table_sizes['qc_size']
    
fec.CORE_AXIS_ENABLE = 63                  # Enable FEC (111111)

Status     Code ID    SC Offset  LA Offset  QC Offset  Code Name 
------     ------     ------     ------     ------     ------    
Loaded     0          0          0          0          docsis_short
Loaded     1          2          2          21         docsis_medium
Loaded     2          4          4          54         docsis_long
Loaded     3          6          6          97         docsis_init_ranging
Loaded     4          8          8          112        docsis_fine_ranging


Choose the appropriate code to perform decoding.

In [6]:
code_name = 'docsis_short'
code_id = ldpc_params.index(code_name)

n = fec._code_params.ldpc[code_name]['n']
k = fec._code_params.ldpc[code_name]['k']
p = fec._code_params.ldpc[code_name]['p']

print('Block Length (bits): %s\nInformation Bits: %s\nSub-Matrix Size: %s' % (n, k, p))

Block Length (bits): 1120
Information Bits: 840
Sub-Matrix Size: 56


The LLR values are for each bit, therefore the buffer length input to the decoder should be the full block size in bits.    
The output from the decoder will only contain the information bits (with the control word we will provide). The RX buffer should be k/8 as the stream width is 8.

In [7]:
K = int(np.ceil(k/8))
N = int(np.ceil(n/8))

tx_buffer = allocate(shape=(n,), dtype=np.uint8)
rx_buffer = allocate(shape=(K,), dtype=np.uint8)

j = 0
for i in range(len(tx_buffer)):
    tx_buffer[i] = llrs_formatted[j]
    j += 1

| Field                          | Type        |
| -----------                    | ----------- |
| External Block ID (31:24)      |  uint8      |
| Max No. Iterations (23:18)     |  uint6      |
| Terminate on no Change (17:17) |  bit1       |
| Terminate on Pass (16:16)      |  bit1       |
| Include Parity Output (15:15)  |  bit1       |
| Hard Output (14:14)            |  bit1       |
| Reserved (13:7)                |  uint7      |
| Code Number (6:0)              |  uint7      |

In [8]:
def non_5G_ldpc_ctrl_dec(id, 
                         max_iterations, 
                         term_on_no_change, 
                         term_on_pass, 
                         include_parity_op,
                         hard_op, 
                         code):
    id = '{0:08b}'.format(id)                               # (31:24) uint8
    max_iterations = '{0:06b}'.format(max_iterations)       # (23:18) uint6
    term_on_no_change = '{0:01b}'.format(term_on_no_change) # (17:17) bit1
    term_on_pass = '{0:01b}'.format(term_on_pass)           # (16:16) bit1
    include_parity_op = '{0:01b}'.format(include_parity_op) # (15:15) bit1
    hard_op = '{0:01b}'.format(hard_op)                     # (14:14) bit1
    reserved = '{0:07b}'.format(0)                          # (13:7) uint7
    code = '{0:07b}'.format(code)                           # (6:0) uint7
    
    print('\tID: ', int(id,2))
    print('\tMax Iter: ', int(max_iterations,2))
    print('\tTerm No Change: ', int(term_on_no_change,2))
    print('\tTerm Pass: ', int(term_on_pass,2))
    print('\tParity Pass: ', int(include_parity_op,2))
    print('\tHard OP: ', int(hard_op,2))
    print('\tCode: ', int(code,2))
    
    ctrl = id + max_iterations + term_on_no_change + term_on_pass \
    + include_parity_op + hard_op + reserved + code
    print('\tBinary: ', ctrl)
    
    return(int(ctrl,2))

ctrl_buffer = allocate(shape=(1,), dtype=np.uint32)

ctrl_buffer[0] = non_5G_ldpc_ctrl_dec(code_id, 24, 1, 1, 0, 1, 0)

	ID:  0
	Max Iter:  24
	Term No Change:  1
	Term Pass:  1
	Parity Pass:  0
	Hard OP:  1
	Code:  0
	Binary:  00000000011000110100000000000000


In [9]:
status_buffer = allocate(shape=(1,), dtype=np.uint32)

Transfer data

In [10]:
dma_data = ol.ldpc_decoder.axi_dma_data
dma_ctrl = ol.ldpc_decoder.axi_dma_ctrl

dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_buffer)
dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_buffer)

dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

Compare the received, decoded data with the initial data that was sent in 02_fec_encoding.

In [11]:
rx_buffer

PynqBuffer([241, 224,  11,  30, 162, 174, 146, 113, 153,  45,  79, 238,
             96,  72, 124, 136, 142, 239, 244,  81, 246, 198, 126,  15,
             30, 213, 187, 119, 222,  66,  60,  21, 117, 200,  57, 193,
             98, 150,  78, 113, 202,  79, 243,  89, 187, 221, 237,  43,
            155, 133,  69,  94, 154,  68, 122,  58, 216,  31, 183, 118,
             52,  29, 134, 102, 250, 200, 252,  36, 111,  97, 161, 124,
            140,  62,  42, 196, 246, 244, 187, 243,  77, 192, 176, 182,
            217,  73,  70,  49, 209, 241,  55,  26, 193,   8, 189, 175,
            214, 177,  97,  17, 207, 196, 156, 251, 112], dtype=uint8)

In [12]:
initial_data

PynqBuffer([241, 224,  11,  30, 162, 174, 146, 113, 153,  45,  79, 238,
             96,  72, 124, 136, 142, 239, 244,  81, 246, 198, 126,  15,
             30, 213, 187, 119, 222,  66,  60,  21, 117, 200,  57, 193,
             98, 150,  78, 113, 202,  79, 243,  89, 187, 221, 237,  43,
            155, 133,  69,  94, 154,  68, 122,  58, 216,  31, 183, 118,
             52,  29, 134, 102, 250, 200, 252,  36, 111,  97, 161, 124,
            140,  62,  42, 196, 246, 244, 187, 243,  77, 192, 176, 182,
            217,  73,  70,  49, 209, 241,  55,  26, 193,   8, 189, 175,
            214, 177,  97,  17, 207, 196, 156, 251, 112], dtype=uint8)

In [13]:
for i in range(len(rx_buffer)):
    if rx_buffer[i] != initial_data[i]:
        print('Error detected at index: %d. %d != %d' % (i, rx_buffer[i], initial_data[i]))